In [1]:
!sh ./recommendation/train.sh

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aiotlab3/.cache/huggingface/token
Login successful
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/aiotlab3/.netrc


In [2]:
import pandas as pd

train_dataset = pd.read_csv('train.csv')
valid_dataset = pd.read_csv('val.csv')
test_dataset = pd.read_csv('test.csv')

In [3]:
from datasets import Dataset, DatasetDict

train_dataset_2 = Dataset.from_csv('train.csv')
valid_dataset_2 = Dataset.from_csv('val.csv')
test_dataset_2 = Dataset.from_csv('test.csv')

dataset = DatasetDict({
    'train': train_dataset_2,
    'validation': valid_dataset_2,
    'test': test_dataset_2,
})


/home/aiotlab3/anaconda3/envs/readsum/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
labels = set()
for label_list in dataset['train']['labels']:
    for label in label_list.split():
        labels.add(label)

for label_list in dataset['validation']['labels']:
    for label in label_list.split():
        labels.add(label)

for label_list in dataset['test']['labels']:
    for label in label_list.split():
        labels.add(label)

labels = sorted(list(labels))  
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
print(len(labels))


31817


In [5]:
def labels_to_one_hot(label_list, label2id):
    num_labels = len(label2id)
    one_hot = np.zeros(num_labels)
    for label in label_list.split():
        label_id = label2id[label]
        one_hot[label_id] = 1
    return one_hot

In [6]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

/home/aiotlab3/anaconda3/envs/readsum/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/aiotlab3/anaconda3/envs/readsum/lib/python3.9/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [7]:
import torch
from torch.utils.data import dataset
from datasets import load_dataset, load_metric
from PIL import Image

class MultiLabelDataset(dataset.Dataset):
    def __init__(self, 
                 df=None, 
                 tokenizer=None, 
                 max_length=1024, 
                 max_samples=None,
                 is_test=False
        ):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test
        
        if max_samples is not None:
            self.df = self.df.sample(n=max_samples, random_state=42).reset_index(drop=True)

    def __len__(self):
        return len(self.df)


    def __getitem__(self, index):
        text = self.df.loc[index, "text"]
        text = text.replace('</s>','<s>')
        labels = self.df.loc[index, "labels"]
        # process caption
        model_inputs = self.tokenizer(
            text, 
            max_length=self.max_length, 
            padding="max_length", 
            truncation=True,
            return_tensors="pt"
        )
        model_inputs = {k:v[0] for k,v in model_inputs.items()}
        
        num_labels = len(label2id)
        one_hot = np.zeros(num_labels)
        for label in labels.split():
            label_id = label2id[label]
            one_hot[label_id] = 1
        
        model_inputs["labels"] = torch.tensor(one_hot, dtype=torch.float)

        return model_inputs

In [8]:
debug = False
train_ds = MultiLabelDataset(
    df=train_dataset,
    tokenizer=tokenizer,
)
val_ds = MultiLabelDataset(
    df=valid_dataset,
    tokenizer=tokenizer,
)
test_ds = MultiLabelDataset(
    df=test_dataset,
    tokenizer=tokenizer,
)


In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

/home/aiotlab3/anaconda3/envs/readsum/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/aiotlab3/anaconda3/envs/readsum/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = './checkpoints',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="wandb",
    run_name='bart-rec'
)

In [11]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [12]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [17]:
# Specify the device
%env CUDA_VISIBLE_DEVICES=$1
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

env: CUDA_VISIBLE_DEVICES=1


In [18]:
trainer.train()

/home/aiotlab3/anaconda3/envs/readsum/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 